In [1]:
from os import getenv
from dotenv import load_dotenv
import json
from services.GoogleSheetsService import *
from services.GeminiService import *
from utils.utils import *
from utils.constants import *

**1. Create a .env file inside credentials/ folder, following the model:**

SHEET_ID={GoogleSheets sheet ID}<br>
RANGE_STRING_INVESTMENTS={page name for investments and range, ex: investment!A1:K999}<br>
RANGE_STRING_APPLICATIONS={page name for applications and range, ex: applications!A1:K999}<br>
RANGE_STRING_RETURNS={page name for returns and range, ex: returns!A1:K999}

**2. Set up Google Gemini API credentials following the instructions in [Google Gemini docs](https://ai.google.dev/gemini-api/docs/quickstart?hl=pt-br&_gl=1*iiivu*_up*MQ..&gclid=CjwKCAjw9IayBhBJEiwAVuc3fu5nCfvDVWPXbLvhT15etQN7YMyqkDg3NFBbBms5iBUHGEp21nPf6RoCFPAQAvD_BwE). Export your api key to environment variables, or put in .env .**

In [2]:
load_dotenv('../credentials/.env')

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = getenv("SHEET_ID")
SHEET_PAGE_INVESTMENTS = getenv("RANGE_STRING_INVESTMENTS")
SHEET_PAGE_APPLICATIONS = getenv("RANGE_STRING_APPLICATIONS")
SHEET_PAGE_RETURNS = getenv("RANGE_STRING_RETURNS")
GEMINI_API_KEY = getenv("GEMINI_API_KEY")

**3. Set up Google Sheets API credentials following the instructions in [Google Cloud docs](https://developers.google.com/sheets/api/quickstart/python?hl=pt-br). Download credentials, rename to secrets.json and put into credentials folder.**

In [3]:
sheetService = GoogleSheetsService(SAMPLE_SPREADSHEET_ID)

# Get sheets based in range strings 
investments = sheetService.get_sheet(SHEET_PAGE_INVESTMENTS).drop('bank', axis=1)
applications = sheetService.get_sheet(SHEET_PAGE_APPLICATIONS).drop('id', axis=1)
returns = sheetService.get_sheet(SHEET_PAGE_RETURNS).drop('bank', axis=1).drop('id', axis=1)

In [4]:
ids = investments['investment_name'].values

In [ ]:
raw_body = {
  'investments': [],
  'applications': [],
  'returns': []
}

# Build a structured json to send to Gemini, dividing investiments in cycles, each cycle can have applications( apply/withdraw money on existing investiments) and returns (record of current value of investment).
for id in ids[3:6]:
  raw_body['investments'].extend(
    restructure_dataframe(
      investments[investments["investment_name"] == id]
    )
  )

  raw_body['applications'].extend(
    restructure_dataframe(
      applications[applications["investment_name"] == id]
    )
  )

  raw_body['returns'].extend(
    restructure_dataframe(
      returns[returns["investment_name"] == id]
    )
  )

print(raw_body)
body_string = f'''Take this as base for the following questions: {json.dumps(raw_body)}'''

In [ ]:
# Initialize Gemini chat and send base prompt, telling how to classify and calculate profit, and sending the database loaded from Google Sheets
chat_bot = GeminiService(GEMINI_API_KEY)
chat_bot.send_message(BASE_PROMPT)
chat_bot.send_message(body_string)

In [ ]:
# Interactive chat
prompt = "Show me a summary of the data provided"
while prompt.lower() != "thanks":
  response = chat_bot.send_message(prompt)
  display(to_markdown(f'**VTR**: {response}'))
  prompt = input("Ask to VTR: ")